In [18]:
import numpy as np

# Bayesian Classifier

In [19]:
import math
# Define a function to calculate the likelihood of a given data point x
# given a normal distribution with specified mean and standard deviation
def pdf(mean, sd, x):
  # To-do
  # vector divide of the following two
  # vector substract, vector divide, vector square, vector multiply
  # vector multiply, vector multiply, vector square root, vector multiply
  exponent = np.multiply(np.square(np.divide(np.subtract(x,mean),sd)),-0.5)
  G = np.sqrt(np.multiply(np.multiply(2.0,np.pi),np.square(sd)))
  likelihood = np.divide(np.exp(exponent),G)
  return likelihood

# Define a function to train a binary Bayes classifier, using one Gaussian for each class
# given a dataset and the number of features (dimensions)
# return the mean, sd for positive and negative samples
def bindary_bc_training(dataset, dim):
  # Initialize variables to store sums and counts for positive and negative samples
  sum_p = np.zeros(dim)
  sum_n = np.zeros(dim)
  sqr_sum_p = np.zeros(dim)
  sqr_sum_n = np.zeros(dim)
  count_p = 0
  count_n = 0

  # Iterate over each sample in the dataset
  for each in range(len(dataset)):
    # Extract the feature values (x_train) and class label (last element)
    x_train=dataset[each][0:dim]
    y_train=dataset[each][dim]
    # If the class label is positive, update the corresponding sums and counts
    if y_train > 0:
      # To-Do
      # vector sum
      # vector square, vector sum
      # scalar add
      if dataset[each][dim] > 0:
        sum_p = np.add(sum_p, x_train)
        sqr_sum_p = np.add(sqr_sum_p, np.multiply(x_train, x_train))
        count_p += 1
    # If the class label is negative, update the corresponding sums and counts
    else :
      # To-Do
      # vector sum
      # vector square, vector sum
      # scalar add
      sum_n = np.add(sum_n, x_train)
      sqr_sum_n = np.add(sqr_sum_n, np.multiply(x_train, x_train))
      count_n += 1

  # Calculate the mean, variance, and standard deviation for positive samples
  # To-Do
  # vector divide
  # vector multiply, vector subtract, vector divide
  # vector sqrt
  mean_p = np.divide(sum_p,count_p)
  variance_p = np.divide(np.subtract(sqr_sum_p,np.multiply(mean_p,sum_p)),count_p-1)
  sd_p = np.sqrt(variance_p)

  # Calculate the mean, variance, and standard deviation for negative samples
  # To-Do
  # vector divide
  # vector multiply, vector subtract, vector divide
  # vector sqrt
  mean_n = np.divide(sum_n,count_n)
  variance_n = np.divide(np.subtract(sqr_sum_n,np.multiply(mean_n,sum_n)),count_n-1)
  sd_n = np.sqrt(variance_n)

  # Return the mean and standard deviation for both classes
  return mean_p, sd_p, mean_n, sd_n

# Define a function to classify a new data point x
# based on the Bayes classifier trained on positive and negative samples
# given a mean and sd for both classes and the prior probabiilies of both classes
# return the class label
def bindary_bc_inferencing(mean_p, sd_p, mean_n, sd_n, x, dim, p_pos, p_neg):
  # Calculate the likelihood of the new data point being in the positive and negative classes
  p_p = pdf (mean_p, sd_p, x)
  p_n = pdf (mean_n, sd_n, x)

  # Compare the product of likelihoods for each class
  # To-Do
  # vector prod, multiply
  # vector prod, multiply
  # If the product for positive class is higher, return 1 (positive class)
  # Otherwise, return -1 (negative class)
  pos_givenx = np.multiply( np.prod( p_p), p_pos)
  neg_given_x = np.multiply( np.prod( p_n), p_neg)
  if pos_givenx > neg_given_x:
    return 1
  else :
    return -1

In [20]:
# Define dimension of the data # To-do
dim = 10

# Define the number of samples to generate
target_num_samples=200 # This is the number of positive samples in the testing set

sum_p, sum_r, sum_f1 = 0,0,0

print("For 10 dimensional space\n")

# Try 30 times
for r in range(30):

  # Initilize the dataset
  Sn = [] # List to store negative samples
  Sp = [] # List to store positive samples
  num_n = 0 # Counter for negative samples
  num_p = 0 # Counter for positive samples
  true_positive = 0  # Counter for true positive samples
  true_negative = 0  # Counter for true negative samples
  false_positive = 0 # Counter for false positive samples
  false_negative = 0 # Counter for false negative samples

  # Generate samples until the target number is reached for both positive and negative samples
  while num_p < target_num_samples*2 or num_n < target_num_samples*2:
    # Generate a random vector within the range [-5, 5] with dimension (dim+1)
    random_sample = np.random.uniform(-5, 5, (dim+1))
    # Set the last dimension to 0 temporally
    random_sample[dim] = 0
    # Compute the sum of the vector components
    sum = np.sum(random_sample)
    # If the sum is positive, append the sample to Sp
    if sum > 0:
      if num_p < target_num_samples*2:
        random_sample[dim] = 1
        Sp.append(random_sample)
        num_p +=1
    elif num_n < target_num_samples*2:
      random_sample[dim] = -1
      Sn.append(random_sample)
      num_n +=1

  # Split the dataset in to training (Sp, Sn) and testing (Tp, Tn)
  # The last target_num_samples samples from Sn and Sp are used for testing.
  Tp = Sp[target_num_samples:]
  Tn = Sn[target_num_samples:]

  # The first target_num_samples samples from Sn and Sp are used for training.
  Sp = Sp[:target_num_samples]
  Sn = Sn[:target_num_samples]

  # Combine positive and negative samples to create the dataset
  dataset = Sn + Sp

  # Calculate the Gausian distribution based on training data
  mean_p, sd_p, mean_n, sd_n = bindary_bc_training(dataset, dim)

  # Combine positive and negative samples to create the dataset
  testing_dataset = Tp + Tn

  # Loop through the testing_dataset
  for each in range(len(testing_dataset)):
    # Make a prediction using the bindary_bc_inferencing function with the testing sample
    prediction = bindary_bc_inferencing(mean_p, sd_p, mean_n, sd_n, testing_dataset[each][0:dim], dim, 0.5, 0.5) # Prior probability = 0.5 for both classes

    # Increment the appropriate counters based on whether the inferencing is true positive, true negative, false positive or false negative
    if testing_dataset[each][dim] == 1:
      if prediction == 1:
        # To-Do
        # increment the appropriate counter
        true_positive+=1
      else :
        # To-Do
        # increment the appropriate counter
        false_negative+=1
    else :
      if prediction == -1:
        # To-Do
        # increment the appropriate counter
        true_negative+=1
      else :
        # To-Do
        # increment the appropriate counter
        false_positive+=1

  # Calculate precision
  # To-Do
  # calculate the precision using the the appropriate counters
  precision = true_positive/(true_positive+false_positive)
  sum_p += precision
  # Calculate recall
  # To-Do
  # calculate the recall using the the appropriate counters
  recall = true_positive / (true_positive + false_negative)
  sum_r += recall
  # Calculate F1 score
  # To-Do
  # calculate the score using precision and recall
  f1score = (2*precision*recall)/(precision+recall)
  sum_f1 += f1score
  # print the result
  print(precision, recall, f1score)

print("\nAverage_precision = ", sum_p/30)
print("Average_recall = ", sum_r/30)
print("Average_f1score = ", sum_f1/30)


For 10 dimensional space

0.9447236180904522 0.94 0.9423558897243106
0.9210526315789473 0.875 0.8974358974358975
0.8699551569506726 0.97 0.9172576832151299
0.9150943396226415 0.97 0.9417475728155339
0.9441624365482234 0.93 0.9370277078085644
0.9068627450980392 0.925 0.9158415841584158
0.8803827751196173 0.92 0.8997555012224938
0.9310344827586207 0.945 0.9379652605459057
0.9257425742574258 0.935 0.9303482587064678
0.9420289855072463 0.975 0.9582309582309583
0.9365079365079365 0.885 0.910025706940874
0.9438775510204082 0.925 0.9343434343434343
0.8695652173913043 0.9 0.8845208845208845
0.93 0.93 0.93
0.927536231884058 0.96 0.9434889434889435
0.9536082474226805 0.925 0.9390862944162438
0.9536082474226805 0.925 0.9390862944162438
0.9036697247706422 0.985 0.9425837320574162
0.9591836734693877 0.94 0.9494949494949495
0.9166666666666666 0.935 0.9257425742574258
0.9396984924623115 0.935 0.93734335839599
0.9306930693069307 0.94 0.9353233830845772
0.8990384615384616 0.935 0.9166666666666667
0.902

In [21]:
# Define dimension of the data # To-do
dim = 2

# Define the number of samples to generate
target_num_samples=40 # This is the number of positive samples in the testing set

sum_p, sum_r, sum_f1 = 0,0,0

print("For 2 dimensional space\n")

# Try 30 times
for r in range(30):

  # Initilize the dataset
  Sn = [] # List to store negative samples
  Sp = [] # List to store positive samples
  num_n = 0 # Counter for negative samples
  num_p = 0 # Counter for positive samples
  true_positive = 0  # Counter for true positive samples
  true_negative = 0  # Counter for true negative samples
  false_positive = 0 # Counter for false positive samples
  false_negative = 0 # Counter for false negative samples

  # Generate samples until the target number is reached for both positive and negative samples
  while num_p < target_num_samples*2 or num_n < target_num_samples*2:
    # Generate a random vector within the range [-5, 5] with dimension (dim+1)
    random_sample = np.random.uniform(-5, 5, (dim+1))
    # Set the last dimension to 0 temporally
    random_sample[dim] = 0
    # Compute the sum of the vector components
    sum = np.sum(random_sample)
    # If the sum is positive, append the sample to Sp
    if sum > 0:
      if num_p < target_num_samples*2:
        random_sample[dim] = 1
        Sp.append(random_sample)
        num_p +=1
    elif num_n < target_num_samples*2:
      random_sample[dim] = -1
      Sn.append(random_sample)
      num_n +=1

  # Split the dataset in to training (Sp, Sn) and testing (Tp, Tn)
  # The last target_num_samples samples from Sn and Sp are used for testing.
  Tp = Sp[target_num_samples:]
  Tn = Sn[target_num_samples:]

  # The first target_num_samples samples from Sn and Sp are used for training.
  Sp = Sp[:target_num_samples]
  Sn = Sn[:target_num_samples]

  # Combine positive and negative samples to create the dataset
  dataset = Sn + Sp

  # Calculate the Gausian distribution based on training data
  mean_p, sd_p, mean_n, sd_n = bindary_bc_training(dataset, dim)

  # Combine positive and negative samples to create the dataset
  testing_dataset = Tp + Tn

  # Loop through the testing_dataset
  for each in range(len(testing_dataset)):
    # Make a prediction using the bindary_bc_inferencing function with the testing sample
    prediction = bindary_bc_inferencing(mean_p, sd_p, mean_n, sd_n, testing_dataset[each][0:dim], dim, 0.5, 0.5) # Prior probability = 0.5 for both classes

    # Increment the appropriate counters based on whether the inferencing is true positive, true negative, false positive or false negative
    if testing_dataset[each][dim] == 1:
      if prediction == 1:
        # To-Do
        # increment the appropriate counter
        true_positive+=1
      else :
        # To-Do
        # increment the appropriate counter
        false_negative+=1
    else :
      if prediction == -1:
        # To-Do
        # increment the appropriate counter
        true_negative+=1
      else :
        # To-Do
        # increment the appropriate counter
        false_positive+=1

  # Calculate precision
  # To-Do
  # calculate the precision using the the appropriate counters
  precision = true_positive/(true_positive+false_positive)
  sum_p += precision
  # Calculate recall
  # To-Do
  # calculate the recall using the the appropriate counters
  recall = true_positive / (true_positive + false_negative)
  sum_r += recall
  # Calculate F1 score
  # To-Do
  # calculate the score using precision and recall
  f1score = (2*precision*recall)/(precision+recall)
  sum_f1 += f1score
  # print the result
  print(precision, recall, f1score)

print("\nAverage_precision = ", sum_p/30)
print("Average_recall = ", sum_r/30)
print("Average_f1score = ", sum_f1/30)


For 2 dimensional space

0.9069767441860465 0.975 0.9397590361445783
0.8695652173913043 1.0 0.9302325581395349
0.8717948717948718 0.85 0.8607594936708861
0.918918918918919 0.85 0.8831168831168831
0.8666666666666667 0.975 0.9176470588235294
0.9473684210526315 0.9 0.9230769230769231
0.8695652173913043 1.0 0.9302325581395349
1.0 0.9 0.9473684210526316
0.8666666666666667 0.975 0.9176470588235294
0.8780487804878049 0.9 0.888888888888889
0.9512195121951219 0.975 0.9629629629629629
0.8 0.9 0.8470588235294118
0.9090909090909091 1.0 0.9523809523809523
1.0 1.0 1.0
0.9459459459459459 0.875 0.9090909090909091
0.9736842105263158 0.925 0.9487179487179489
0.9473684210526315 0.9 0.9230769230769231
0.975609756097561 1.0 0.9876543209876543
0.8974358974358975 0.875 0.8860759493670887
0.975 0.975 0.975
1.0 1.0 1.0
0.8666666666666667 0.975 0.9176470588235294
0.9736842105263158 0.925 0.9487179487179489
0.975609756097561 1.0 0.9876543209876543
0.8222222222222222 0.925 0.8705882352941177
0.9069767441860465 0.